In [1]:
from seeq import spy
import pandas as pd

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 193

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.pull

Retrieves data for the specified items over a given time range.

Once you have searched for signals, conditions or scalars and have a DataFrame with an `ID` column, you can execute `spy.pull` to retrieve data for those items.

Alternatively, you can execute `spy.pull('worksheet_url')` if you want to pull all the items loaded in an Analysis 
worksheet by supplying the URL of the Seeq worksheet. You can copy the value of worksheet_url directly out of your 
browser's address bar without further modification.

## Time interval

You must specify `start` and `end`, which can be any data type or format that is recognized by the Pandas function `pandas.to_datetime()`.

Let's create a DataFrame of signals to retrieve data for:

In [3]:
items = spy.search({
    'Path': 'Example >> Cooling Tower 1 >> Area A',
    'Name': 'Compressor'
})

items

,Path,Name,Time,Count,Pages,Result
0,Example >> Cooling Tower 1 >> Area A,Compressor,00:00:00.07,2,1,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,0EF58FC2-8762-F910-A3F7-157C1D6689FB,Example >> Cooling Tower 1,Area A,Compressor Power,NaN,StoredSignal,kW,Example Data,False
1,0EF58FC2-8CFB-EAC0-A6FA-3B586A7AEA13,Example >> Cooling Tower 1,Area A,Compressor Stage,NaN,StoredSignal,string,Example Data,False


Now we will retrieve that data for a specified time interval:

In [4]:
spy.pull(items, start='2019-01-01', end='2019-01-07')

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.09,577,1,69 KB,Success
1,0EF58FC2-8CFB-EAC0-A6FA-3B586A7AEA13,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Stage,00:00:00.13,577,1,69 KB,Success


,Example >> Cooling Tower 1 >> Area A >> Compressor Power,Example >> Cooling Tower 1 >> Area A >> Compressor Stage
2019-01-01 00:00:00+00:00,39.189300,STAGE 2
2019-01-01 00:15:00+00:00,39.842881,STAGE 2
2019-01-01 00:30:00+00:00,39.949065,STAGE 2
2019-01-01 00:45:00+00:00,39.424393,STAGE 2
2019-01-01 01:00:00+00:00,39.326496,STAGE 2
...,...,...
2019-01-06 23:00:00+00:00,16.848487,STAGE 1
2019-01-06 23:15:00+00:00,16.870912,STAGE 1
2019-01-06 23:30:00+00:00,16.829447,STAGE 1
2019-01-06 23:45:00+00:00,17.096314,STAGE 1


The index of the returned DataFrame is `pd.Timestamp` objects with the timezone that is set for the workstation or server that is running Jupyter. Each signal is in a separate column.

Alternatively, you can request a specific timezone for both the input and output (and they can be different):

In [5]:
spy.pull(items,
         start=pd.Timestamp('2019-01-01', tz='US/Pacific'),
         end=pd.Timestamp('2019-01-07', tz='US/Pacific'),
         tz_convert='US/Eastern')

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.05,577,1,12 KB,Success
1,0EF58FC2-8CFB-EAC0-A6FA-3B586A7AEA13,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Stage,00:00:00.05,577,1,12 KB,Success


,Example >> Cooling Tower 1 >> Area A >> Compressor Power,Example >> Cooling Tower 1 >> Area A >> Compressor Stage
2019-01-01 03:00:00-05:00,0.002923,OFF
2019-01-01 03:15:00-05:00,0.002923,OFF
2019-01-01 03:30:00-05:00,0.002923,OFF
2019-01-01 03:45:00-05:00,0.002924,OFF
2019-01-01 04:00:00-05:00,0.002923,OFF
...,...,...
2019-01-07 02:00:00-05:00,0.002923,OFF
2019-01-07 02:15:00-05:00,0.002923,OFF
2019-01-07 02:30:00-05:00,0.002923,OFF
2019-01-07 02:45:00-05:00,0.002923,OFF


If you are pulling items from an Analysis worksheet URL, `start` and `end` are selected  by default from the start 
and end of the Display Range in the Trend view of the worksheet. 

`worksheet_url = 'http://localhost:34216/workbook/<workbookID>/worksheet/<worksheetID>'`

`spy.pull(url)`

However, you can always specify a different `start` or `end` when executing `spy.pull` 

`spy.pull(worksheet_url, start='2019-01-01', end='2019-01-07')`

## Gridding

By default, SPy Pull will interpolate data onto a _grid_ at 15 minute intervals. This is often desirable for computation within matrices, and Seeq goes to great lengths to ensure that interpolation is done correctly.

You can change spacing by specifying a `grid` parameter with units of `s`, `min`, `h` or `d`. For example, `grid='1h'` will cause samples to be spaced every hour. For dense data, specifying a larger grid interval can significantly reduce the amount of data transferred to Python, which can reduce time taken to pull data.

You can also specify a time zone for alignment: `grid="12h, 'Canada/Pacific'"`. To align samples to an offset, include
both a timezone and a date or timestamp formatted as YYYY-MM-DD,
YYYY-MM-DDTHH:MM:SS, or ISO8601. For example, to align samples every day at every 7:17 AM
and 7:17 PM in Vancouver, even across daylight savings time changes: `grid="12h,'Canada/Pacific','2020-06-01T07:17:00'"`.

Alternatively, specifying `grid='auto'` will automatically estimate a grid value. The grid can be determined in either of two ways:

1. If the column `Estimated Sample Period` exists in the query DataFrame, the grid value will be the median of the 
sample periods in the query DataFrame. Note that in this case the sample period is determined for the specified time 
period issued by `spy.search(estimate_sample_period=dict(Start='<sampling_start_time>', 
End='<sampling_end_time>'))`.
2. If the column `Estimated Sample Period` does not exist in the query DataFrame, the grid will be determined by running the `estimateSamplePeriod()` Formula function on each signal in the query DataFrame for the specified time period with the `start` and `end` parameters and then taking the median of all the sample periods. It is highly recommended to estimate the sample period using `spy.search` first and to validate the estimates since the time taken to pull data can significantly increase for dense data and queries with a large number of signals. 

All signals in Seeq honor their `Maximum Interpolation` property. If the gap between two adjacent samples is larger than `Maximum Interpolation`, then no interpolation is performed and you may see `NaN` entries in the DataFrame, which indicates that there is no appropriate value for that cell. This concept of `Maximum Interpolation` has both a functional purpose and performance implications. It can prevent a process variable like a temperature reading from "flatlining" inappropriately if no data is collected for a period of time. It is also constrains the amount of data that must be retrieved from a source system in order to interpolate correctly to the edges of a request interval.

Specifying `grid=None` retrieves the data with _raw_ timestamps and no interpolation. On rows where signals don't happen to all have values at a particular timestamp, you will see `NaN` entries.

**NaN vs None**: `spy.pull()` uses `NaN` to represent a _missing value_, consistent with Pandas idioms. `None`, on other hand, represents an _invalid_ value, which in Seeq Workbench will break interpolation if one or more invalid samples are located between two valid samples.

## Column headers

By default, column headers will contain the full path to the item (if it's part of an asset tree) or just the name of the item (if it's not in an asset tree).

You can force the column headers to be populated with a particular property using the `header` argument. In this example, the column header will use just the `Name` property:

In [6]:
spy.pull(items, start='2019-01-01T12:00:00', end='2019-01-01T13:00:00', header='Name')

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.01,5,1,80 B,Success
1,0EF58FC2-8CFB-EAC0-A6FA-3B586A7AEA13,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Stage,00:00:00.02,5,1,80 B,Success


,Compressor Power,Compressor Stage
2019-01-01 12:00:00+00:00,0.002923,OFF
2019-01-01 12:15:00+00:00,0.002971,OFF
2019-01-01 12:30:00+00:00,0.002923,OFF
2019-01-01 12:45:00+00:00,0.002923,OFF
2019-01-01 13:00:00+00:00,0.002923,OFF


Alternatively, you can supply a `Header` column in the `items` DataFrame whose values indicate the column headers you would like to be used for each item that is pulled.

## Applying a Seeq Formula

You can leverage Seeq's time series calculation engine easily by adding a `calculation` argument to `spy.pull()`. For example, let's say we want calculate an hourly average and represent it as a step-interpolated signal. You can figure out what to put as the `calculation` argument by experimenting in Seeq Workbench and going to the _Item Properties_ pane to look at the formula for a particular calculated item in Seeq. Then you copy and paste that _Formula_ property into your notebook, taking care to use either `$signal`, `$condition` or `$scalar` (as appropriate) for the variable name.

Now when we fetch the data, it will have the formula applied to the output:

In [7]:
signal_with_calc = spy.search({'Name': 'Area A_Temperature', 'Datasource Name': 'Example Data'})

spy.pull(signal_with_calc,
         start='2019-01-01T00:00:00',
         end='2019-01-01T03:00:00',
         calculation='$signal.aggregate(average(), hours(), startKey())',
         grid=None)

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-8206-77F0-8519-ECD251B886A6,StoredSignal,Area A_Temperature,00:00:00.13,4,1,4 KB,Success


,Area A_Temperature
2019-01-01 00:00:00+00:00,95.605355
2019-01-01 01:00:00+00:00,96.929129
2019-01-01 02:00:00+00:00,97.176269
2019-01-01 03:00:00+00:00,96.347881


Alternatively, you can supply a `Calculation` column in the `items` DataFrame and specify the formula for each item independently. Here we specify both a `Calculation` and a `Header` column to retrieve the same signal with different formulas:

In [8]:
area_a_rel_humidity = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': 'Area A',
    'Name': 'Relative Humidity',
    'Datasource Name': 'Example Data'
})
two_calcs = pd.concat([area_a_rel_humidity, area_a_rel_humidity]).reset_index()
two_calcs['Calculation'] = pd.Series([
    '$signal.aggregate(average(), hours(), startKey())',
    '$signal.aggregate(average(), periods(2h), startKey())'
])
two_calcs['Header'] = pd.Series([
    '1-hour Average',
    '2-hour Average'
])

spy.pull(two_calcs, start='2019-01-01T00:00:00', end='2019-01-01T05:00:00', grid=None)

,ID,Type,Path,Asset,Name,Header,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-7E6E-7160-ACE8-E1D5408B23E1,CalculatedSignal,Example >> Cooling Tower 1,Area A,Relative Humidity,1-hour Average,00:00:00.06,6,1,5 KB,Success
1,0EF58FC2-7E6E-7160-ACE8-E1D5408B23E1,CalculatedSignal,Example >> Cooling Tower 1,Area A,Relative Humidity,2-hour Average,00:00:00.08,3,1,6 KB,Success


,1-hour Average,2-hour Average
2019-01-01 00:00:00+00:00,40.243403,38.644717
2019-01-01 01:00:00+00:00,37.046031,NaN
2019-01-01 02:00:00+00:00,34.629467,34.165053
2019-01-01 03:00:00+00:00,33.700638,NaN
2019-01-01 04:00:00+00:00,33.695556,34.327242
2019-01-01 05:00:00+00:00,34.958928,NaN


## Applying a Seeq Formula Across Assets

If you have an asset tree (say, from OSIsoft Asset Framework), you can apply a calculation across assets as long as those assets have the necessary attributes (signals, conditions or scalars) with identical names.

To get started, it's easiest to create the calculation in Seeq Workbench using a single asset first. You can "swap" to other assets by finding an asset in the Data tab and clicking the "swap" button. In this example, we're going to use spy.push() to create a calculated signal for use later.

In [9]:
area_a_signals = spy.search({
    'Path': 'Example >> Cooling Tower 1 >> Area A'
})

dew_point_calc = spy.push(metadata=pd.DataFrame([{
    'Type': 'Signal',
    'Name': 'Dew Point',
    # From https://iridl.ldeo.columbia.edu/dochelp/QA/Basic/dewpoint.html
    'Formula': "$T - ((100 - $RH.setUnits(''))/5)",
    'Formula Parameters': {
        '$T': area_a_signals[area_a_signals['Name'] == 'Temperature'],
        '$RH': area_a_signals[area_a_signals['Name'] == 'Relative Humidity']
    }
}]), workbook='SPy Documentation Examples >> spy.pull')

dew_point_calc

,Type,Name,Formula,Formula Parameters,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,CalculatedSignal,Dew Point,$T - ((100 - $RH.setUnits(''))/5),"[RH=0EF58FC2-7E6E-7160-ACE8-E1D5408B23E1, T=0E...",0EF5BF13-0348-F910-81FC-319F8D739929,Seeq Data Lab,Seeq Data Lab,[0EF5BF13-0348-F910-81FC-319F8D739929] {Signal...,0EF5BF13-06B9-EEA0-ABBB-0A9F1F9D60FB,Success


Now you can search for all the assets you want to apply this calculation to, and use the `calculation` argument in `spy.pull()` to retrieve data for the various assets.

In [10]:
all_areas = spy.search({
    'Path': 'Example >> Cooling Tower 1'
}, recursive=False)

spy.pull(all_areas, calculation=dew_point_calc)

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-90AC-77F0-85F2-91128B10372A,CalculatedSignal,Example,Cooling Tower 1,Area A,00:00:00.52,4,1,992 B,Success
1,0EF58FC2-90AA-60E0-94F7-29D4E89DC2F9,CalculatedSignal,Example,Cooling Tower 1,Area B,00:00:00.52,4,1,992 B,Success
2,0EF58FC2-90AA-60E0-846C-0711E3BCB468,CalculatedSignal,Example,Cooling Tower 1,Area C,00:00:00.51,4,1,992 B,Success
3,0EF58FC2-90AC-77F0-ADA9-570BE8320694,CalculatedSignal,Example,Cooling Tower 1,Area G,00:00:00.52,4,1,992 B,Success
4,0EF58FC2-90AA-60E0-914C-BA4C89AD8CDB,CalculatedSignal,Example,Cooling Tower 1,Area H,00:00:00.51,4,1,992 B,Success
5,0EF58FC2-90AA-60E0-AC5F-539EFD1C4108,CalculatedSignal,Example,Cooling Tower 1,Area I,00:00:00.50,4,1,992 B,Success
6,0EF58FC2-90A7-F9D0-A519-4BB719820A3F,CalculatedSignal,Example,Cooling Tower 1,Area J,00:00:00.51,4,1,992 B,Success
7,0EF58FC2-90A7-F9D0-90F1-BA89D4852085,CalculatedSignal,Example,Cooling Tower 1,Area K,00:00:00.49,4,1,992 B,Success


,Example >> Cooling Tower 1 >> Area A,Example >> Cooling Tower 1 >> Area B,Example >> Cooling Tower 1 >> Area C,Example >> Cooling Tower 1 >> Area G,Example >> Cooling Tower 1 >> Area H,Example >> Cooling Tower 1 >> Area I,Example >> Cooling Tower 1 >> Area J,Example >> Cooling Tower 1 >> Area K
2024-08-16 16:15:00+00:00,50.798338,54.140696,49.810592,64.241942,56.713197,65.494262,59.615899,71.098165
2024-08-16 16:30:00+00:00,50.634327,53.236205,49.476706,63.574725,56.619293,65.506675,59.687074,71.020064
2024-08-16 16:45:00+00:00,50.308455,53.015179,49.728350,63.712202,56.341640,65.220495,59.567813,71.045078
2024-08-16 17:00:00+00:00,50.136478,52.452233,49.614424,63.946073,56.032253,64.870019,58.423700,71.137121


## Conditions

Conditions capture time intervals of interest as _capsules_, which have a start time and an end time. They will not have an end time if they are _uncertain_ (aka _in progress_), and they may not have a start or end time if it lies outside of the time interval (see below for more information).

In order to illustrate how condition data can be pulled, we must first push a simple condition into Seeq. (Don't worry-- this condition will be scoped to a workbook that only you can see.)

In [11]:
compressor_on_high = spy.push(metadata=pd.DataFrame([{
    'Name': 'Compressor on High',
    'Type': 'Condition',

    'Formula Parameters': {
        # Note here that we are just using a row from our search results. The SPy module will figure
        # out that it contains an identifier that we can use.
        '$cp': items[items['Name'] == 'Compressor Power']
    },

    # This formula specifies capsules to be created when Compressor Power is above 25 kW
    'Formula': '$cp.valueSearch(isGreaterThan(25kW))'
}]), workbook='SPy Documentation Examples >> spy.pull')

compressor_on_high

,Name,Type,Formula Parameters,Formula,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,Compressor on High,CalculatedCondition,[cp=0EF58FC2-8762-F910-A3F7-157C1D6689FB],$cp.valueSearch(isGreaterThan(25kW)),0EF5BF13-0348-F910-81FC-319F8D739929,Seeq Data Lab,Seeq Data Lab,[0EF5BF13-0348-F910-81FC-319F8D739929] {Condit...,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,Success


We can now pull data for a particular time period for that condition:

In [12]:
spy.pull(compressor_on_high, start='2019-01-01T04:00:00Z', end='2019-01-09T02:00:00Z')

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,Compressor on High,00:00:00.13,9,1,91 KB,Success


,Condition,Capsule Start,Capsule End,Capsule Is Uncertain
0,Compressor on High,NaT,2019-01-01 05:01:31.176600146+00:00,False
1,Compressor on High,2019-01-02 00:16:37.331031739+00:00,2019-01-02 05:00:33.238973103+00:00,False
2,Compressor on High,2019-01-03 00:13:18.731947840+00:00,2019-01-03 02:55:03.027869976+00:00,False
3,Compressor on High,2019-01-03 17:28:04.251451273+00:00,2019-01-03 18:21:58.603314005+00:00,False
4,Compressor on High,2019-01-04 17:20:33.782090068+00:00,2019-01-04 18:46:41.571969511+00:00,False
5,Compressor on High,2019-01-05 02:18:11.253715409+00:00,2019-01-05 05:01:23.108876186+00:00,False
6,Compressor on High,2019-01-05 17:20:29.256597642+00:00,2019-01-05 18:43:39.192264349+00:00,False
7,Compressor on High,2019-01-07 17:21:18.826407749+00:00,2019-01-07 18:40:09.983726541+00:00,False
8,Compressor on High,2019-01-09 00:13:55.007784046+00:00,NaT,False


If there were multiple conditions, the `Condition` field would show other conditions further down.

Notice that the first capsule (at index 0) has no start time. That's because it starts outside the supplied time interval. Similarly, the last capsule (at index 8) has no end time because it ends outside the supplied time interval.

`Capsule Is Uncertain` will be `True` for capsules whose exit criteria has not yet been satisfied and may change as a result of new data arriving in the system. In Seeq Workbench, such capsules are "hollowed out" in the Gantt bars at the top of the trend.

If we want to know the start and end time even if they're outside our time interval, we can specify a `calculation` argument that applies the `removeLongerThan()` function, which allows the Seeq calculation engine to retrieve start/end times.

For example:

In [13]:
spy.pull(compressor_on_high,
         start='2019-01-01T04:00:00Z',
         end='2019-01-09T02:00:00Z',
         calculation='$condition.removeLongerThan(1d)')

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,Compressor on High,00:00:00.10,9,1,23 KB,Success


,Condition,Capsule Start,Capsule End,Capsule Is Uncertain
0,Compressor on High,2018-12-31 17:19:18.871250820+00:00,2019-01-01 05:01:31.176600146+00:00,False
1,Compressor on High,2019-01-02 00:16:37.331031739+00:00,2019-01-02 05:00:33.238973103+00:00,False
2,Compressor on High,2019-01-03 00:13:18.731947840+00:00,2019-01-03 02:55:03.027869976+00:00,False
3,Compressor on High,2019-01-03 17:28:04.251451273+00:00,2019-01-03 18:21:58.603314005+00:00,False
4,Compressor on High,2019-01-04 17:20:33.782090068+00:00,2019-01-04 18:46:41.571969511+00:00,False
5,Compressor on High,2019-01-05 02:18:11.253715409+00:00,2019-01-05 05:01:23.108876186+00:00,False
6,Compressor on High,2019-01-05 17:20:29.256597642+00:00,2019-01-05 18:43:39.192264349+00:00,False
7,Compressor on High,2019-01-07 17:21:18.826407749+00:00,2019-01-07 18:40:09.983726541+00:00,False
8,Compressor on High,2019-01-09 00:13:55.007784046+00:00,2019-01-09 04:29:12.815103591+00:00,False


Now the first and last capsules have both starts and ends.

It's important to note that setting the _Maximum Duration_ for a Condition to something high can have major performance implications because the Seeq calculation engine is required to potentially evaluate significant amounts of data before/after the query interval to "find" the start/end of the capsule.

### Conditions as signals

You can also request the condition data to be represented as a signal with a value of `1` if a capsule is present at a particular timestamp or `0` if not. Use `shape='samples'` like so:

In [14]:
spy.pull(compressor_on_high, start='2019-01-01T00:00:00Z', end='2019-01-01T12:00:00Z', shape='samples', grid='1h')

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,Compressor on High,00:00:00.02,1,1,128 B,Success


,Compressor on High
2019-01-01 00:00:00+00:00,1.0
2019-01-01 01:00:00+00:00,1.0
2019-01-01 02:00:00+00:00,1.0
2019-01-01 03:00:00+00:00,1.0
2019-01-01 04:00:00+00:00,1.0
2019-01-01 05:00:00+00:00,1.0
2019-01-01 06:00:00+00:00,0.0
2019-01-01 07:00:00+00:00,0.0
2019-01-01 08:00:00+00:00,0.0
2019-01-01 09:00:00+00:00,0.0


### Mixing conditions with signals

If the DataFrame that you pass into the `spy.pull()` function contains a mix of signals and conditions, `shape='samples'` will be automatically applied:

In [15]:
compressor_signal_and_condition = pd.concat([spy.search({
    'Path': 'Example >> Cooling Tower 1 >> Area A',
    'Name': 'Compressor Power',
    'Datasource Name': 'Example Data'
}), spy.search({
    'Name': 'Compressor*High',
}, workbook='SPy Documentation Examples >> spy.pull')], sort=False, ignore_index=True)

spy.pull(compressor_signal_and_condition,
         start='2019-01-01T00:00:00Z', end='2019-01-01T12:00:00Z',
         header='Name', grid='1h')

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.02,13,1,5 KB,Success
1,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,nan,nan,Compressor on High,00:00:00.01,1,1,128 B,Success


,Compressor Power,Compressor on High
2019-01-01 00:00:00+00:00,39.189300,1.0
2019-01-01 01:00:00+00:00,39.326496,1.0
2019-01-01 02:00:00+00:00,40.223175,1.0
2019-01-01 03:00:00+00:00,39.431483,1.0
2019-01-01 04:00:00+00:00,39.139759,1.0
2019-01-01 05:00:00+00:00,39.154395,1.0
2019-01-01 06:00:00+00:00,0.002923,0.0
2019-01-01 07:00:00+00:00,0.002923,0.0
2019-01-01 08:00:00+00:00,0.002923,0.0
2019-01-01 09:00:00+00:00,0.002923,0.0


You can supply `shape='capsules'` to force the output to capsules. In such a case, the signals within the DataFrame will be aggregated within the interval of each capsule according to a statistic. By default, that statistic is `average`, but you can add a `Statistic` column to the passed-in DataFrame to control which statistic is used. To return multiple statistics for the same signal, add additional signal rows to your DataFrame that only vary by the `Statistic` column. The possible statistic function names you can specify are documented in Seeq Workbench's Formula tool documentation under the heading _Signal Value Statistics_.

Let's try this example:

In [16]:
area_a_compressor_power = spy.search({
    'Path': 'Example >> Cooling Tower 1 >> Area A',
    'Name': 'Compressor Power',
    'Datasource Name': 'Example Data'
}, quiet=True)

compressor_on_high = spy.search({
    'Name': 'Compressor*High',
}, workbook='SPy Documentation Examples >> spy.pull', quiet=True).iloc[0]

area_a_compressor_power_max = area_a_compressor_power.iloc[0].copy()
area_a_compressor_power_max['Statistic'] = 'maximum'

area_a_compressor_power_delta = area_a_compressor_power.iloc[0].copy()
area_a_compressor_power_delta['Statistic'] = 'totalized'

conditions_with_stats = pd.DataFrame([
    compressor_on_high,
    area_a_compressor_power_max,
    area_a_compressor_power_delta
]).reset_index(drop=True)

conditions_with_stats

,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Path,Asset,Statistic
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,Compressor on High,NaN,CalculatedCondition,NaN,Seeq Data Lab,False,NaN,NaN,NaN
1,0EF58FC2-8762-F910-A3F7-157C1D6689FB,Compressor Power,NaN,StoredSignal,kW,Example Data,False,Example >> Cooling Tower 1,Area A,maximum
2,0EF58FC2-8762-F910-A3F7-157C1D6689FB,Compressor Power,NaN,StoredSignal,kW,Example Data,False,Example >> Cooling Tower 1,Area A,totalized


Notice that the `Compressor Power` signal appears twice in this DataFrame but has a different value in the `Statistic` column. If we pull with this DataFrame and the `shape='capsules'` argument, we'll get columns for these aggregates:

In [17]:
pull_results = spy.pull(conditions_with_stats,
                        start='2019-01-01T00:00:00Z', end='2019-01-07T00:00:00Z',
                        shape='capsules', header='Name', grid='1h')

# For brevity, drop uncertainty column
pull_results.drop(columns=['Capsule Is Uncertain'])

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,nan,nan,Compressor on High,00:00:00.21,7,1,43 KB,Success
1,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.21,7,1,43 KB,Success
2,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,00:00:00.21,7,1,43 KB,Success


,Condition,Capsule Start,Capsule End,Compressor Power (maximum),Compressor Power (totalized)
0,Compressor on High,NaT,2019-01-01 05:01:31.176600146+00:00,NaN,NaN
1,Compressor on High,2019-01-02 00:16:37.331031739+00:00,2019-01-02 05:00:33.238973103+00:00,37.337571,621836.085754
2,Compressor on High,2019-01-03 00:13:18.731947840+00:00,2019-01-03 02:55:03.027869976+00:00,38.346637,355158.928285
3,Compressor on High,2019-01-03 17:28:04.251451273+00:00,2019-01-03 18:21:58.603314005+00:00,34.499000,109815.164071
4,Compressor on High,2019-01-04 17:20:33.782090068+00:00,2019-01-04 18:46:41.571969511+00:00,35.749304,181575.891856
5,Compressor on High,2019-01-05 02:18:11.253715409+00:00,2019-01-05 05:01:23.108876186+00:00,39.769103,378094.380825
6,Compressor on High,2019-01-05 17:20:29.256597642+00:00,2019-01-05 18:43:39.192264349+00:00,35.700364,174524.224726


## Scalars

Scalars represent a constant value across all time. Let's push one into Seeq to see how it looks when we retrieve it:

In [18]:
compressor_power_limit = spy.push(metadata=pd.DataFrame([{
    'Name': 'Compressor Power Limit',
    'Type': 'Scalar',
    'Formula': '50kW'
}]), workbook='SPy Documentation Examples >> spy.pull', errors='raise')

compressor_power_limit

,Name,Type,Formula,Scoped To,Datasource Class,Datasource ID,Formula Parameters,Data ID,ID,Push Result
0,Compressor Power Limit,CalculatedScalar,50kW,0EF5BF13-0348-F910-81FC-319F8D739929,Seeq Data Lab,Seeq Data Lab,[],[0EF5BF13-0348-F910-81FC-319F8D739929] {Scalar...,0EF5BF13-322C-FF60-9C3D-A7BBBF38E2E6,Success


In [19]:
spy.pull(compressor_power_limit)

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-322C-FF60-9C3D-A7BBBF38E2E6,CalculatedScalar,Compressor Power Limit,00:00:00.01,1,0,0 B,Success


,Compressor Power Limit
0,50


## Pickling
Sometimes a pull is "expensive" and takes a long time to complete, and the data likely hasn't changed.
In that case, it is common to "pickle" the resulting DataFrame for later use, like so:

In [20]:
pull_results.to_pickle('pickled_pull.pkl')

Now you can "unpickle" the DataFrame and use it without incurring the cost of re-executing the pull:

In [21]:
unpickled_pull = pd.read_pickle('pickled_pull.pkl')

You can access some of the original context of the search via the `spy` attribute of the unpickled
DataFrame. For example, the `spy.status` tells you what queries were successful and what queries failed.

In [22]:
unpickled_pull.spy.status.df

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0EF5BF13-2032-EE90-B3F9-EF5D74B1E75D,CalculatedCondition,NaN,NaN,Compressor on High,0 days 00:00:00.213521,7,1,43 KB,Success
1,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,0 days 00:00:00.213521,7,1,43 KB,Success
2,0EF58FC2-8762-F910-A3F7-157C1D6689FB,StoredSignal,Example >> Cooling Tower 1,Area A,Compressor Power,0 days 00:00:00.213521,7,1,43 KB,Success


## Performance Metrics

If your pull operation takes a long time to complete, you can take a look at the timing information for data processing by inspecting the "Data Processed" cell in the status DataFrame like so:

In [23]:
# Pull two months of data
pull_results = spy.pull(spy.search({'Name': 'Area B_Compressor Power'}), start='2019-01-01', end='2019-03-01', grid=None)

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0EF58FC2-85A1-7590-AA5F-C66CA82059F8,StoredSignal,Area B_Compressor Power,00:00:00.37,42481,1,679 KB,Success


In [24]:
# Inspect the 'Data Processed' column in the first row of the status DataFrame
pull_results.spy.status.df.iloc[0]['Data Processed']

Time spent in Request Queue:           0:00:00.000520
Time spent reading Metadata:           0:00:00.003270
Time spent waiting for Datasource(s):  0:00:00
Time spent reading from Seeq Cache:    0:00:00
Time spent in Calc Engine Queue:       0:00:00
Time spent in Calc Engine:             0:00:00.291160
Time spent reclaiming Memory:          0:00:00

Metadata items read:                                3
Metadata relationships read:                        0
Samples read from Datasource(s):                42482
Samples read from Persistent Cache:                 0
Samples read from In-Memory Cache:                  0
Capsules read from Datasource(s):                   0
Capsules read from Persistent Cache:                0
Capsules read from In-Memory Cache:                 0
Total bytes processed:                         679712

Note that the "Time spent" values here are inclusive of the time spent within the Seeq service. They are _exclusive_ of time spent transmitting the data from the Seeq service to SPy, and the time spent by SPy organizing the data into a Pandas DataFrame.

## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [25]:
help(spy.pull)

Help on function pull in module seeq.spy._pull:

pull(items, *, start=None, end=None, grid='15min', header='__auto__', group_by=None, shape: 'Union[str, Callable]' = 'auto', capsule_properties=None, tz_convert=None, calculation=None, bounding_values=False, invalid_values_as=nan, enums_as='string', errors=None, quiet=None, status: 'Status' = None, session: 'Optional[Session]' = None, capsules_as=None)
    Retrieves signal, condition or scalar data from Seeq Server and returns it
    in a DataFrame.
    
    Parameters
    ----------
    items : {str, pd.DataFrame, pd.Series}
        A DataFrame or Series containing ID and Type columns that can be used
        to identify the items to pull. This is usually created via a call to
        spy.search(). Alternatively, you can supply URL of a Seeq Workbench
        worksheet as a str.
    
        If a 'Calculation' column is present, the formula specified in that
        column will be applied to the item in that row "on-the-fly" while
     